In [1]:
# Load the MMLU dataset
from datasets import load_dataset
import pandas as pd
df_easy = pd.DataFrame([])
df_hard = pd.DataFrame([])
categories = ['boolean_expressions', 'causal_judgement', 'date_understanding', 'disambiguation_qa', 'dyck_languages', 'formal_fallacies', 'geometric_shapes', 'hyperbaton', 'logical_deduction_five_objects', 'logical_deduction_seven_objects', 'logical_deduction_three_objects', 'movie_recommendation', 'multistep_arithmetic_two', 'navigate', 'object_counting', 'penguins_in_a_table', 'reasoning_about_colored_objects', 'ruin_names', 'salient_translation_error_detection', 'snarks', 'sports_understanding', 'temporal_sequences', 'tracking_shuffled_objects_five_objects', 'tracking_shuffled_objects_seven_objects', 'tracking_shuffled_objects_three_objects', 'web_of_lies', 'word_sorting']
for c in categories:
    try:
        dataset = load_dataset("tasksource/bigbench",c)
        data = pd.DataFrame(dataset['train'])
        data['Category'] = c
        df_easy = pd.concat([df_easy,data])
    except:
        continue
    dataset = load_dataset("maveriq/bigbenchhard",c)
    data = pd.DataFrame(dataset['train'])
    data['Category'] = c
    df_hard = pd.concat([df_hard,data])

/home/guangya/miniconda3/envs/dl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/guangya/miniconda3/envs/dl/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for tasksource/bigbench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tasksource/bigbench
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/guangya/miniconda3/envs/dl/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for tasksource/bigbench contains custom code which must be executed to correctly load the dataset. Y

In [2]:
def is_valid_target(value):
    if isinstance(value, str):
        value = value.strip()
        if len(value) == 1 and (value.isalpha()):
            return True
        if value.upper() in ['YES', 'NO']:
            return True
    return False
df_hard['target'] = df_hard['target'].str.replace(r'\(|\)', '', regex=True)
# Filter the data based on the target values
filtered_data = df_hard[df_hard['target'].apply(is_valid_target)]

In [3]:
filtered_data['target'] = filtered_data['target'].str.upper()

/tmp/ipykernel_29394/3495407291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['target'] = filtered_data['target'].str.upper()


In [4]:
df_hard = filtered_data

In [5]:
df_hard['Name'] = 'BigBench_hard'
columms_interest = df_hard.Category.unique()
columms_interest

array(['date_understanding', 'disambiguation_qa', 'geometric_shapes',
       'hyperbaton', 'movie_recommendation', 'navigate',
       'penguins_in_a_table', 'reasoning_about_colored_objects',
       'ruin_names', 'salient_translation_error_detection', 'snarks',
       'sports_understanding', 'temporal_sequences'], dtype=object)

In [6]:
# 2: Rename the columns
df_hard.rename(columns={'input': 'Question', 'target': 'Correct Answer'}, inplace=True)


In [7]:
df_easy = df_easy[df_easy['Category'].isin(columms_interest)]

In [8]:
import re
def convert_choices(input_str):
    choices = re.findall(r'choice: (.+)', input_str)
    if not choices:
        return input_str

    options = '\nOptions:'
    for i, choice in enumerate(choices, start=ord('A')):
        options += f'\n({chr(i)}) {choice}'

    output_str = re.sub(r'choice: .+', '', input_str, flags=re.DOTALL)
    output_str += options

    return output_str
df_easy = df_easy[~df_easy['inputs'].str.startswith('Q:')]
df_easy['inputs'] = df_easy['inputs'].apply(convert_choices)

In [9]:
columms_interest = df_easy.Category.unique()

In [26]:
def convert_target_to_string(target):
    if isinstance(target, list) and len(target) > 0:
        return target[0]
    return target

df_easy['targets'] = df_easy['targets'].apply(convert_target_to_string)

In [38]:
def convert_target(row):
    target = row['targets']
    if target in ['True', 'False']:
        return 'Yes' if target == 'True' else 'No'
    else:
        target = row['multiple_choice_scores']
        if len(target) == 1:
            return target[0]
        elif len(target) > 1:
            max_score_index = target.index(max(target))
            return chr(65 + max_score_index)  # Convert index to corresponding letter (A, B, C, ...)
    return str(target)

# Assuming your DataFrame is named df_easy with columns 'targets' and 'multiple_choice_scores'
df_easy['converted_targets'] = df_easy.apply(convert_target, axis=1)

In [41]:
df_easy.drop(columns=['targets', 'multiple_choice_scores','multiple_choice_targets','idx'], inplace=True)

In [43]:
df_easy['Name'] = 'BigBench_easy'

In [44]:
df_easy

,inputs,Category,converted_targets,Name
0,"In the following sentences, explain the antece...",disambiguation_qa,A,BigBench_easy
1,"In the following sentences, explain the antece...",disambiguation_qa,C,BigBench_easy
2,"In the following sentences, explain the antece...",disambiguation_qa,C,BigBench_easy
3,"In the following sentences, explain the antece...",disambiguation_qa,C,BigBench_easy
4,"In the following sentences, explain the antece...",disambiguation_qa,C,BigBench_easy
...,...,...,...,...
795,"Today, William went to the orchestra hall. Bet...",temporal_sequences,A,BigBench_easy
796,"Today, Michael went to the clothing store. Bet...",temporal_sequences,A,BigBench_easy
797,"Today, Jason went to the gas station. Between ...",temporal_sequences,A,BigBench_easy
798,"Today, Lisa went to the bakery. Between what t...",temporal_sequences,A,BigBench_easy


In [45]:
df_hard

,Question,Correct Answer,Category,Name
0,"In the following sentences, explain the antece...",A,disambiguation_qa,BigBench_hard
1,"In the following sentences, explain the antece...",C,disambiguation_qa,BigBench_hard
2,"In the following sentences, explain the antece...",C,disambiguation_qa,BigBench_hard
3,"In the following sentences, explain the antece...",C,disambiguation_qa,BigBench_hard
4,"In the following sentences, explain the antece...",C,disambiguation_qa,BigBench_hard
...,...,...,...,...
245,"Today, Elizabeth went to the swimming pool. Be...",C,temporal_sequences,BigBench_hard
246,"Today, Lisa went to the art show. Between what...",C,temporal_sequences,BigBench_hard
247,"Today, Nancy went to the movies. Between what ...",A,temporal_sequences,BigBench_hard
248,"Today, John went to the clothing store. Betwee...",A,temporal_sequences,BigBench_hard


In [46]:
df_hard = df_hard[df_hard['Category'].isin(df_easy['Category'].unique())]
df_hard.to_csv('Bigbench_hard.csv',index = False)
df_easy.to_csv('Bigbench_easy.csv',index = False)

df_all = pd.concat([df_easy,df_hard])
df_all.to_csv('../preprocessed/Bigbench.csv',index = False)